In [ ]:
import os
import pandas as pd
import joblib
from transformers import AutoTokenizer, AutoModel

from custom_modules.predict import classify_texts

Concatenate dfs

In [ ]:
# concatenate excels from same folder

DIR_OF_PROCESSED_NEWS = r"G:\temp\msds_temp\Test_new_bert_news"

filelist=[]
for path, subdirs, files in os.walk(DIR_OF_PROCESSED_NEWS):
    for file in files:
        if file.endswith('.xlsx'):
            filelist.append(os.path.join(path, file))
dfs = []
for f in filelist[:-1]:
    dfs.append(pd.read_excel(f))
df = pd.concat(dfs)

#df.to_excel('concatenated_news.xlsx')

Forecast of news impact

In [ ]:
# Load tokenizer and model from Hugging Face Hub
model_name = "cointegrated/rubert-tiny2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# load pretrained regressors
regress_knn = joblib.load('pkl_models/regress_knn.pkl')
regress_svr = joblib.load('pkl_models/regress_svr.pkl')

In [6]:
outcomes_knn = classify_texts(df['text'], regress_knn, tokenizer = tokenizer, model =model)

100%|██████████| 4429/4429 [02:43<00:00, 27.07it/s]


In [7]:
outcomes_svr = classify_texts(df['text'], regress_svr, tokenizer = tokenizer, model =model)

100%|██████████| 4429/4429 [01:36<00:00, 45.76it/s]


In [10]:
df['knn_forecast'] = outcomes_knn
df['svr_forecast'] = outcomes_svr

In [ ]:
# final forecasts
df

,text_id,text,caption,date,id,time,source,source_id,securityid,securityid_count,score,firms_count,sent_module,negative_proba,neutral_proba,positive_proba,knn_forecast,svr_forecast
0,-1001087766403_64589_1,Совет директоров «Роснефти» решил возобновить ...,NaN,2024-04-01,64589,17:44:47,https://t.me/forbesrussia,-1001087766403,ROSN,4,0.937500,1,0.818860,0.167718,0.181140,0.651141,0.200000,0.341314
1,-1001087766403_64585_1,«Сбер» объявил об обновлении бонусной программ...,NaN,2024-04-01,64585,17:17:05,https://t.me/forbesrussia,-1001087766403,SBER,1,1.000000,1,0.827519,0.026612,0.172481,0.800907,0.335584,0.508028
2,-1001087766403_64576_1,Мосбиржа включит акции МКПАО «Яндекс» в список...,NaN,2024-04-01,64576,15:36:57,https://t.me/forbesrussia,-1001087766403,YNDX,1,1.000000,1,0.389876,0.032023,0.610124,0.357853,0.600000,0.828280
3,-1001087766403_64565_1,Мосбиржа решила приостановить торги расписками...,NaN,2024-04-01,64565,13:15:00,https://t.me/forbesrussia,-1001087766403,FIVE,1,1.000000,1,0.889141,0.724939,0.110859,0.164203,0.328224,0.527101
4,-1001087766403_64555_1,Совкомбанк получил разрешения ЦБ и ФАС на поку...,NaN,2024-04-01,64555,10:45:52,https://t.me/forbesrussia,-1001087766403,SVCB,4,0.976190,1,0.667462,0.252594,0.332538,0.414868,0.128840,0.611733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26,-1001052157402_43713_1,"Аэрофлот сообщил, что, по состоянию на 17:00 м...",NaN,2024-07-06,43713,18:01:14,https://t.me/BFMnews,-1001052157402,AFLT,4,1.000000,1,0.929811,0.491765,0.070189,0.438046,0.406243,0.822790
27,-1001052157402_43709_1,"❗️Для стабилизации расписания ""Аэрофлот"" отмен...",NaN,2024-07-06,43709,15:24:24,https://t.me/BFMnews,-1001052157402,AFLT,4,1.000000,1,0.856886,0.496140,0.143114,0.360746,1.000000,0.837846
28,-1001052157402_43707_1,В Ростове - на - Дону вводятся веерные отключе...,NaN,2024-07-06,43707,13:52:30,https://t.me/BFMnews,-1001052157402,FEES,1,1.000000,2,0.667379,0.633133,0.332621,0.034246,1.000000,0.822870
29,-1001052157402_43707_1,В Ростове - на - Дону вводятся веерные отключе...,NaN,2024-07-06,43707,13:52:30,https://t.me/BFMnews,-1001052157402,MRKY,2,1.000000,2,0.667379,0.633133,0.332621,0.034246,1.000000,0.822870


In [ ]:
# Save
#df.to_excel('knn_svr_real_test_1.xlsx')